<a href="https://colab.research.google.com/github/leah-apking/project3.1/blob/main/PKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install & Import Dependencies

In [6]:
pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-six3_mac
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-six3_mac
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done


In [7]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 21.1 MB/s eta 0:00:00


In [8]:
!python -m spacy download en_core_web_sm

2023-10-20 13:01:39.726630: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
import pandas as pd
import numpy as np
import pke
from transformers import pipeline

## Pull NPS Feedback Data

In [10]:
month='09'
nps_path = f"{month}.ClientNPSSurveyFeedback.csv"
nps_data = pd.read_csv(nps_path)

In [11]:
nps = nps_data[['FullName', 'LoanNumber', 'CompanyName', 'NetPromoterLabel', 'Rating',
       'Comments', 'PerformedBy']]
nps1 = nps.rename(columns={'FullName':'ClientName', 'PerformedBy':'TeamMemberName'})
nps1["Comment"] = nps1.Comments.map(lambda x: 'False' if x == '0' else 'True')

In [12]:
nps2 = nps1.loc[nps1['Comment'] == 'True']
comments = nps2[['LoanNumber', 'Comments']]
comments1 = comments.dropna()

In [80]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_comments = comments.loc[comments['WordCount'] > 10]


<ipython-input-80-db387d62d77f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


,LoanNumber,Comments,WordCount
2,1.223512e+09,I needed a last minute change early in the mor...,40.0
6,1.223501e+09,"Jackie Berry and Christina Plante, were awesom...",17.0
8,1.223432e+09,There were a couple of hiccups in UW but was a...,15.0
13,1.523106e+09,Slam Dunk Baby! We closed fast as usual and we...,54.0
23,1.223498e+09,Briana Kent went above and beyond as an u/w. V...,55.0


## Extract Sentiment and KeyPhrases from Comments

In [61]:
phraselist= []
scorelist = []

def pke_extract(text):
  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  keyphrases = extractor.get_n_best(n=10, stemming=False)

  for i, (candidate, score) in enumerate(keyphrases):

    phraselist.append(candidate)
    scorelist.append(score)

for n in range(len(nps_comments)):
    text = nps_comments.iloc[n,1]
    pke_extract(text)

In [62]:
list1 = []
sentimentlist = []

def sentiment(text):

  sentiment_pipeline = pipeline("sentiment-analysis")

  sentiments = sentiment_pipeline(text)

  for s, data in enumerate(sentiments):
    list1.append(s)
    sentimentlist.append(data)

for n in range(len(nps_comments)):
    text = nps_comments.iloc[n,1]
    sentiment(text)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-englis

In [63]:
sentiment_df = pd.DataFrame(sentimentlist)
# sentiment_df

In [71]:
sentimentlist = []
list1 = []

ranklist = []
phraselist= []
scorelist = []

def sentiment(text):

  sentiment_pipeline = pipeline("sentiment-analysis")

  sentiments = sentiment_pipeline(text)

  for s, data in enumerate(sentiments):
    list1.append(s)
    sentimentlist.append(data)

def pke_extract(text):
  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  keyphrases = extractor.get_n_best(n=10, stemming=False)

  for i, (candidate, score) in enumerate(keyphrases):

    ranklist.append(i)
    phraselist.append(candidate)
    scorelist.append(score)

for n in range(len(nps_comments)):
    text = nps_comments.iloc[n,1]
    sentiment(text)
    pke_extract(text)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-englis

## Build DataFrame

In [72]:
sentiment_df = pd.DataFrame(sentimentlist)
sentiment_df.head()

,label,score
0,POSITIVE,0.999792
1,NEGATIVE,0.999359
2,NEGATIVE,0.997751
3,POSITIVE,0.999782
4,POSITIVE,0.999818


In [75]:
def divide_chunks(l, n):

    for i in range(0, len(l), n):
        yield l[i:i + n]
n = 5

phrases = list(divide_chunks(phraselist, n))
columns = ['1st', '2nd', '3rd', '4th', '5th']
keywords_df = pd.DataFrame(phrases,columns=columns)
keywords_df

,1st,2nd,3rd,4th,5th
0,questions,loan submission,closing team,help,cortez
1,able,payoff,beneficial,system,thing
2,complicated,actual uw,server,nice,way
3,smooth process,income pre-underwritten,scenario lopp,loan,amazing
4,tyra thomas-spratley,issue,day,transparent,uwm
5,professional,start,experience,efficient process,loan fun
6,team,business partners,clients,great partnership,uwm
7,loan coordinator,aaron burch,fantastic,future,uwm
8,world,best lo,flawless experience,easy,actual uw
9,server,nice,way,smooth process,income pre-underwritten


In [79]:
extracted_df = pd.concat([nps_comments, sentiment_df, keywords_df], axis=1)
extracted_df

,label,score,label,score,1st,2nd,3rd,4th,5th
0,POSITIVE,0.999792,POSITIVE,0.999792,questions,loan submission,closing team,help,cortez
1,NEGATIVE,0.999359,NEGATIVE,0.999359,able,payoff,beneficial,system,thing
2,NEGATIVE,0.997751,NEGATIVE,0.997751,complicated,actual uw,server,nice,way
3,POSITIVE,0.999782,POSITIVE,0.999782,smooth process,income pre-underwritten,scenario lopp,loan,amazing
4,POSITIVE,0.999818,POSITIVE,0.999818,tyra thomas-spratley,issue,day,transparent,uwm
5,POSITIVE,0.999863,POSITIVE,0.999863,professional,start,experience,efficient process,loan fun
6,POSITIVE,0.999847,POSITIVE,0.999847,team,business partners,clients,great partnership,uwm
7,POSITIVE,0.999853,POSITIVE,0.999853,loan coordinator,aaron burch,fantastic,future,uwm
8,NEGATIVE,0.997751,NEGATIVE,0.997751,world,best lo,flawless experience,easy,actual uw
9,POSITIVE,0.998642,POSITIVE,0.998642,server,nice,way,smooth process,income pre-underwritten
